# Automatic Differentiation
AD is an extremely powerful tool. In julia, you can differentiate almost any valid julia program to obtain derivatives, gradients, jacobians and hessians etc. automatically, with high performance. This is what makes up the bulk of most deep-learning libraries, but contrary to most libraries, you do not need to write your code using a subset of julia or a DL-specific language, you can just write regular julia code.

There are a number of different kinds of AD. In the following, we will refer to a function 
$$ f : \mathbb{R}^n -> \mathbb{R}^m$$

## Forward-mode AD
Using dual numbers, forward-mode AD perfrorms a single forward pass of your program, calculating both the function value and gradients in one go. FAD is algorithmically favorable when $f$ is "few to many", or $n < m$. It also typically has the least overhead, so is competetive when both $n$ and $m$ are small.

## Reverse-mode AD
This is what is used in DL libraries. RAD works by constructing a computation graph, either before execution (as old tensorflow) or during the execution (most common today).
RAD is algorithmically favorable when $f$ is "many to few", or $n > m$. This is the case in most DL, where the cost function is a scalar-valued function of very many parameters, the NN weights. For functions with many outputs, 

In [1]:
using ForwardDiff, BenchmarkTools

f(x) = sum(sin, x) + prod(tan, x) * sum(sqrt, x);

x = rand(5) # small size 
g = x -> ForwardDiff.gradient(f, x); # g = ∇f
g(x)

5-element Array{Float64,1}:
 3.5149762945487666
 2.286481415170909 
 1.9477252312769016
 1.9630995451261493
 2.1217540210288757

In [2]:
@btime g($x);

  567.685 ns (3 allocations: 688 bytes)


In [3]:
ForwardDiff.hessian(f, x)

5×5 Array{Float64,2}:
 1.94011  6.11041  5.47095  5.44969  5.65504
 6.11041  1.49091  3.08014  3.06812  3.18352
 5.47095  3.08014  2.25584  2.74717  2.8506 
 5.44969  3.06812  2.74717  2.11657  2.83948
 5.65504  3.18352  2.8506   2.83948  1.60844

In [4]:
using Zygote

gz = x -> Zygote.gradient(f, x)[1]; # g = ∇f
gz(x) ≈ g(x)

true

In [5]:
@btime gz($x);

  24.050 μs (321 allocations: 11.19 KiB)


In [6]:
using Flux.Tracker
using Flux.Tracker: data

gf = x -> data(Tracker.gradient(f, x)[1]) # g = ∇f
gf(x) ≈ g(x)

true

In [7]:
@btime gf($x);

  8.700 μs (143 allocations: 5.18 KiB)


If we change the size of the input vector, the relative timings change

In [8]:
x = rand(5000)
@btime g($x);

  106.736 ms (5 allocations: 548.17 KiB)


In [9]:
#@btime gz($x);

In [10]:
@btime gf($x);

  364.825 μs (157 allocations: 551.40 KiB)


Most AD (except for Zygote and perhaps Yota) in julia works by either overloading `Base` functions on custom types. This means that you can not use AD if you restrict the input types to your functions too much! In the following example, the input is restricted to `Vector{Float64}`

In [11]:
x = abs.(randn(3))
f2(x::Vector{Float64}) = sum(sin, x) + prod(tan, x) * sum(sqrt, x);
f2(x)

9.681237039644286

In [12]:
ForwardDiff.gradient(f2, x);

MethodError: MethodError: no method matching f2(::Array{ForwardDiff.Dual{ForwardDiff.Tag{typeof(f2),Float64},Float64,3},1})
Closest candidates are:
  f2(!Matched::Array{Float64,1}) at In[11]:2

This did not work, since ForwardDiff  calls the function with an argument of type `Vector{<: Dual}`

In [13]:
Tracker.gradient(f2, x);

MethodError: MethodError: no method matching f2(::TrackedArray{…,Array{Float64,1}})
Closest candidates are:
  f2(!Matched::Array{Float64,1}) at In[11]:2

This didn't work either, since `Tracker` calls the function with an argument of type `TrackedArray`.

In [14]:
f3(x::Vector) = sum(sin, x) + prod(tan, x) * sum(sqrt, x);
ForwardDiff.gradient(f2, x)

MethodError: MethodError: no method matching f2(::Array{ForwardDiff.Dual{ForwardDiff.Tag{typeof(f2),Float64},Float64,3},1})
Closest candidates are:
  f2(!Matched::Array{Float64,1}) at In[11]:2

In [15]:
Tracker.gradient(f3, x);

MethodError: MethodError: no method matching f3(::TrackedArray{…,Array{Float64,1}})
Closest candidates are:
  f3(!Matched::Array{T,1} where T) at In[14]:1